### 1. Import Libraries and Loading Dataset

In [1]:
import os
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 

import plotly.express as px
from scipy.stats import skew
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error,mean_absolute_error

%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
data_dir = "/content/drive/MyDrive/zillow-prize-1"

# Read Property Information from 2016 and 2017
properties_2016 = pd.read_csv( data_dir +"/properties_2016.csv", low_memory=False)
properties_2017 = pd.read_csv( data_dir +"/properties_2017.csv", low_memory=False)

# Read Training Information
train_2016  = pd.read_csv( data_dir +"/train_2016_v2.csv", low_memory=False)
train_2017  = pd.read_csv( data_dir +"/train_2017.csv", low_memory=False)

# Read Submission Data
sample_submission_data = pd.read_csv(data_dir+'/sample_submission.csv', low_memory = False)

#merge training with properties
train_2016 = pd.merge(train_2016, properties_2016, how = 'left', on = 'parcelid').copy()
train_2017 = pd.merge(train_2017, properties_2017, how = 'left', on = 'parcelid').copy()

train_df = pd.concat([train_2016,train_2017],axis=0)
original_train_df=train_df
train_df # Merge Dataset from 2016 and 2017 for Train Data

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.027600,2016-01-01,1.0,NaN,NaN,2.0,3.0,NaN,4.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.168400,2016-01-01,NaN,NaN,NaN,3.5,4.0,NaN,NaN,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,-0.004000,2016-01-01,1.0,NaN,NaN,3.0,2.0,NaN,4.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,0.021800,2016-01-02,1.0,NaN,NaN,2.0,2.0,NaN,4.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,-0.005000,2016-01-02,NaN,NaN,NaN,2.5,4.0,NaN,NaN,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77608,10833991,-0.002245,2017-09-20,1.0,NaN,NaN,3.0,3.0,NaN,8.0,...,NaN,NaN,265000.0,379000.0,2016.0,114000.0,4685.34,NaN,NaN,6.037132e+13
77609,11000655,0.020615,2017-09-20,NaN,NaN,NaN,2.0,2.0,NaN,6.0,...,NaN,NaN,70917.0,354621.0,2016.0,283704.0,4478.43,NaN,NaN,6.037101e+13
77610,17239384,0.013209,2017-09-21,NaN,NaN,NaN,2.0,4.0,NaN,NaN,...,1.0,NaN,50683.0,67205.0,2016.0,16522.0,1107.48,NaN,NaN,6.111008e+13
77611,12773139,0.037129,2017-09-21,1.0,NaN,NaN,1.0,3.0,NaN,4.0,...,NaN,NaN,32797.0,49546.0,2016.0,16749.0,876.43,NaN,NaN,6.037434e+13


### 2. Data Preprocessing Stage

In [3]:
#replace nan 
def replace_nan(data): 
  print("Replace Empty Values with 0")
  print()
  data = data.fillna(0) 
  #test_df = test_df.fillna(0)   
  return data 
 
#one_hot_encoding 
def one_hot_encoding(train_filtered): 
  categorical_features = ['airconditioningtypeid', 'heatingorsystemtypeid', 'propertylandusedesc', 'storytypeid', 'architecturalstyletypeid', 'typeconstructiontypeid', 'buildingclasstypeid']

  print("Perform One hot encoding")
  print()

  for i in categorical_features:
    if i in train_filtered.columns:
      feature_df = pd.DataFrame(train_filtered, columns=[i]) 
      dum_df = pd.get_dummies(feature_df, columns=[i], prefix=[ i + "_"]) 
      train_filtered = pd.concat([train_filtered, dum_df], axis = 1) 
      train_filtered = train_filtered.drop(columns=[i])   
   
  return train_filtered

def clean_data(data):
  print("Cleaning Data")
  print()


  dataframe = data.copy()

  
  dataframe['property_tax_per_sqft'] = dataframe['taxamount'] / dataframe['calculatedfinishedsquarefeet']
  dataframe['missing_finished_area'] = dataframe['calculatedfinishedsquarefeet'].isnull().astype(np.float32)


  empty_pool_index = dataframe[dataframe["poolcnt"]==0].index
  dataframe.loc[empty_pool_index,'poolsizesum'] = 0

  pool_index = dataframe[(dataframe['poolcnt'] > 0) & (dataframe['poolsizesum'].isnull())].index
  dataframe.loc[pool_index,'poolsizesum'] = np.nan

  dataframe['derived_room_cnt'] = dataframe['bedroomcnt'] + dataframe['bathroomcnt']

  #bad_index = dataframe[dataframe.unitcnt.isnull()].index   
  #dataframe.loc[bad_index,'unitcnt'] = 1
  dataframe['location_1'] = dataframe['latitude'] + dataframe['longitude']
  dataframe['location_2'] = dataframe['latitude'] - dataframe['longitude']
  dataframe['location_3'] = dataframe['latitude'] + 0.5 * dataframe['longitude']
  dataframe['location_4'] = dataframe['latitude'] - 0.5 * dataframe['longitude']

  dataframe['avg_garage_size'] = dataframe['garagetotalsqft'] / dataframe['garagecarcnt']

  dataframe['fireplacecnt'] = np.where(dataframe['fireplacecnt'].isnull(),0,1)

  dataframe['decktypeid'] = np.where(dataframe['decktypeid'].isnull(),0,1)

  dataframe['taxdelinquencyflag'] = np.where(dataframe['taxdelinquencyflag']=='',0,1)
  dataframe['derived_room_cnt'] = dataframe['bedroomcnt'] + dataframe['bathroomcnt'] 
  dataframe['hashottuborspa'] = np.where(dataframe['hashottuborspa']=='',0,1)
  
  #dataframe['airconditioningtypeid'] = np.where(dataframe['airconditioningtypeid'].isnull(),0,1)
  #dataframe['heatingorsystemtypeid'] = np.where(dataframe['heatingorsystemtypeid'].isnull(),0,1)

  #dataframe['airconditioningtypeid'] = np.where((dataframe["airconditioningtypeid"].isnull()) & (dataframe.heatingorsystemtypeid==2),1,dataframe["airconditioningtypeid"])

  #dataframe.loc[dataframe[dataframe["heatingorsystemtypeid"].isnull()].index,'heatingorsystemtypeid']=0
  

  invalid_garages = dataframe.loc[dataframe['calculatedfinishedsquarefeet']<dataframe['finishedfloor1squarefeet']].index
  dataframe.loc[invalid_garages,'finishedfloor1squarefeet'] = np.nan

  garage_index = dataframe[(dataframe.garagecarcnt==0) & (dataframe['garagetotalsqft']>0)].index
  dataframe.loc[garage_index,'garagecarcnt'] = np.nan

  #bad_index = dataframe[dataframe['taxvaluedollarcnt'].isnull()].index
  #dataframe.loc[bad_index,'taxvaluedollarcnt'] = dataframe.loc[bad_index,'structuretaxvaluedollarcnt'] + dataframe.loc[bad_index,'landtaxvaluedollarcnt']
  
  return dataframe


def transform_date(X): 
  print("Transforming Date Information")
  print()
  dt = pd.to_datetime(X['transactiondate']).dt
  X['transaction_year'] = dt.year
  #X['transaction_month'] = ((dt.year - 2016)*12 + dt.month).astype('category')
  X['transaction_month'] = dt.month
  #X['transaction_day'] = dt.day
  #X['transaction_quarter'] = ((dt.year - 2016)*4 + dt.quarter).astype('category')
  X = X.drop(['transactiondate'], axis=1)

  return X


def columns_to_remove(df, threshold):
  print("Removing Columns with" + str(threshold) + "% missing data")
  columns_to_drop = []
  columns_to_keep = []

  percent_missing = train_df.isnull().sum() * 100 / len(train_df)
  missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
  missing_value_df.sort_values('percent_missing', inplace=True,ascending=False)

  print(missing_value_df)

  # Example: 100 means 100 % of the data is missing while 20 means 20% of the data is missing
  #fig = px.bar(missing_value_df,x=missing_value_df.index.values, y=missing_value_df.percent_missing,title='Missing Information')
  #fig.show()

  for index, row in missing_value_df.iterrows():
      if(row['percent_missing']>threshold):
        columns_to_drop.append(index)
      else:
        columns_to_keep.append(index)

  print("Columns to Drop",len(columns_to_drop),columns_to_drop)
  print("Columns to Keep",len(columns_to_keep),columns_to_keep)
  print()

  df = df.drop(columns_to_drop,axis=1)

  return df,columns_to_drop

def select_features_correlate_positive(train_df):
  features_corr = train_df.corr()['logerror'].sort_values(ascending=False).reset_index().rename(columns={'index':'feature','logerror':'correlation'})
  positive_feature = features_corr[features_corr.correlation>0]
  negative_feature = features_corr[(features_corr.correlation<0) & (features_corr.correlation>-0.001)]

  cols = [col for col in positive_feature.feature if col not in ['logerror','parcelid']]
  neg_cor = [col for col in negative_feature.feature if col not in ['logerror','parcelid']]

  print(features_corr)
  cols = cols +['propertycountylandusecode']#'propertyzoningdesc',
  print('we have about {} selected features'.format(len(cols)))
  return cols, neg_cor

def data_processing(df):
  df = clean_data(df)
  df, columns_to_drop = columns_to_remove(df,87)
  df = one_hot_encoding(df)
  df = replace_nan(df) 
  df = transform_date(df)
  return df,columns_to_drop


### Feature Selection

In [ ]:
# cols, neg_cor = select_features_correlate_positive(train_df)
# print(neg_cor)
# print(cols)

In [4]:
train_df = original_train_df.copy()
# train_df = train_df.drop(neg_cor,axis=1)
train_df, columns_to_drop_test = data_processing(train_df)
#train_df

Cleaning Data

Removing Columns with87% missing data
                              percent_missing
buildingclasstypeid                 99.981535
finishedsquarefeet13                99.955327
storytypeid                         99.944606
basementsqft                        99.944606
yardbuildingsqft26                  99.901720
fireplaceflag                       99.765320
architecturalstyletypeid            99.721243
typeconstructiontypeid              99.689078
finishedsquarefeet6                 99.519322
decktypeid                          99.242352
pooltypeid10                        99.031497
poolsizesum                         98.905223
pooltypeid2                         98.643143
hashottuborspa                      97.674640
taxdelinquencyyear                  97.210640
taxdelinquencyflag                  97.210640
yardbuildingsqft17                  96.998594
finishedsquarefeet15                96.074168
finishedfloor1squarefeet            92.320476
finishedsquarefeet50       

### 4. Splitting Data for Training Data

In [5]:
print(train_df.shape)

(167888, 64)


In [6]:
def prepare_dataset(df, columns_to_drop, test_size, random_state):    
     
    # divide data to train and test
    df_train, df_test = train_test_split(df, test_size=test_size, random_state=random_state)
    
    # scale the training inputs
    x_train = df_train.drop(columns_to_drop,axis=1)
    y_train = df_train['logerror'].to_numpy()

    x_test = df_test.drop(columns_to_drop,axis=1)
    y_test = df_test['logerror'].to_numpy() 
    
    return x_train, y_train, x_test, y_test

In [7]:
additonal_columns_to_drop= ['logerror', 'propertycountylandusecode', 'parcelid', 'propertyzoningdesc']
X_train, y_train, X_test, y_test = prepare_dataset(train_df, additonal_columns_to_drop, 0.2,20)

In [8]:
X_train

,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,garagecarcnt,garagetotalsqft,...,heatingorsystemtypeid__10.0,heatingorsystemtypeid__11.0,heatingorsystemtypeid__12.0,heatingorsystemtypeid__13.0,heatingorsystemtypeid__14.0,heatingorsystemtypeid__18.0,heatingorsystemtypeid__20.0,heatingorsystemtypeid__24.0,transaction_year,transaction_month
66756,3.0,4.0,7.0,3.0,2698.0,2698.0,6037.0,3.0,0.0,0.0,...,0,0,0,0,0,0,0,0,2017,8
3808,2.0,2.0,4.0,2.0,1463.0,1463.0,6037.0,2.0,0.0,0.0,...,0,0,0,0,0,0,0,0,2016,1
941,1.5,2.0,0.0,1.5,993.0,993.0,6059.0,1.0,2.0,0.0,...,0,0,0,0,0,0,0,0,2017,1
22922,2.0,4.0,0.0,2.0,1430.0,1430.0,6111.0,2.0,1.0,390.0,...,0,0,0,0,0,0,0,0,2016,4
36824,3.0,4.0,4.0,3.0,1740.0,0.0,6037.0,3.0,0.0,0.0,...,0,0,0,0,0,0,0,0,2017,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34033,1.0,0.0,6.0,1.0,426.0,426.0,6037.0,1.0,0.0,0.0,...,0,0,0,0,0,0,0,0,2017,5
31962,2.0,2.0,0.0,2.0,975.0,975.0,6059.0,2.0,1.0,0.0,...,0,0,0,0,0,0,0,0,2016,5
23775,2.0,3.0,7.0,2.0,1098.0,1098.0,6037.0,2.0,0.0,0.0,...,0,0,0,0,0,0,0,0,2016,4
37135,2.0,3.0,4.0,2.0,1410.0,1410.0,6037.0,2.0,0.0,0.0,...,0,0,0,0,0,0,0,0,2016,5


In [ ]:
#X_train

5. Model Training

In [9]:
X_train.dtypes

bathroomcnt                     float64
bedroomcnt                      float64
buildingqualitytypeid           float64
calculatedbathnbr               float64
calculatedfinishedsquarefeet    float64
finishedsquarefeet12            float64
fips                            float64
fullbathcnt                     float64
garagecarcnt                    float64
garagetotalsqft                 float64
latitude                        float64
longitude                       float64
lotsizesquarefeet               float64
poolcnt                         float64
pooltypeid7                     float64
propertylandusetypeid           float64
rawcensustractandblock          float64
regionidcity                    float64
regionidcounty                  float64
regionidneighborhood            float64
regionidzip                     float64
roomcnt                         float64
threequarterbathnbr             float64
unitcnt                         float64
yearbuilt                       float64


### Recursive Feature Elimination

In [ ]:
# # evaluate RFE for regression
# from numpy import mean
# from numpy import std
# from sklearn.datasets import make_regression
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import RepeatedKFold
# from sklearn.feature_selection import RFE
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import AdaBoostRegressor
# from sklearn.pipeline import Pipeline

In [ ]:
# rfe = RFE(estimator=AdaBoostRegressor())
# rfe=rfe.fit(X_train, y_train)

In [ ]:
# important_features_rfe = rfe.get_support(1) #the most important features


In [ ]:
# print(important_features_rfe)
# print(len(important_features_rfe))

In [ ]:
# rfe_columns=[]
# for i in important_features_rfe:
#   rfe_columns.append(X_train.columns[i])

# print(rfe_columns)
# print(len(rfe_columns))

In [ ]:
# #using randomforest features
# #rfe_columns = ['bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid', 'calculatedbathnbr', 'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'fullbathcnt', 'garagetotalsqft', 'latitude', 'longitude', 'lotsizesquarefeet', 'propertylandusetypeid', 'rawcensustractandblock', 'regionidcity', 'regionidneighborhood', 'regionidzip', 'roomcnt', 'yearbuilt', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt', 'taxamount', 'censustractandblock', 'transaction_year', 'transaction_month']
# X_train = X_train.loc[:, rfe_columns]
# X_train

### Catboost

In [10]:
!pip install catboost
from catboost import CatBoostRegressor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
catBoost_models = []
for i in range(5):
  #catBoost = CatBoostRegressor(loss_function='MAE', eval_metric='MAE', random_seed = i)
  catBoost = CatBoostRegressor(loss_function='MAE', eval_metric='MAE', learning_rate = 0.15, depth = 4, random_seed = i)
  catBoost.fit(X_train,y_train)
  catBoost_models.append(catBoost)

0:	learn: 0.0685643	total: 76.1ms	remaining: 1m 16s
1:	learn: 0.0685197	total: 103ms	remaining: 51.5s
2:	learn: 0.0684725	total: 128ms	remaining: 42.4s
3:	learn: 0.0684371	total: 154ms	remaining: 38.3s
4:	learn: 0.0684044	total: 177ms	remaining: 35.3s
5:	learn: 0.0683726	total: 204ms	remaining: 33.7s
6:	learn: 0.0683467	total: 262ms	remaining: 37.2s
7:	learn: 0.0683190	total: 302ms	remaining: 37.4s
8:	learn: 0.0682978	total: 347ms	remaining: 38.2s
9:	learn: 0.0682731	total: 414ms	remaining: 41s
10:	learn: 0.0682567	total: 481ms	remaining: 43.2s
11:	learn: 0.0682429	total: 544ms	remaining: 44.8s
12:	learn: 0.0682263	total: 607ms	remaining: 46s
13:	learn: 0.0682165	total: 631ms	remaining: 44.4s
14:	learn: 0.0682070	total: 656ms	remaining: 43.1s
15:	learn: 0.0681991	total: 682ms	remaining: 41.9s
16:	learn: 0.0681877	total: 705ms	remaining: 40.8s
17:	learn: 0.0681783	total: 730ms	remaining: 39.8s
18:	learn: 0.0681660	total: 759ms	remaining: 39.2s
19:	learn: 0.0681584	total: 785ms	remaining

In [12]:
catBoost_models

In [13]:
#X_test_2=X_test.loc[:, rfe_columns]
for i in catBoost_models:
  y_pred= i.predict(X_test)
  rmse = mean_squared_error(y_test, y_pred, squared=False)
  mae = mean_absolute_error(y_test, y_pred)
  print(f"Model: {i}")
  print(f"MAE: {mae}, RMSE: {rmse}")

Model: <catboost.core.CatBoostRegressor object at 0x7f83a19b1790>
MAE: 0.06880667577688462, RMSE: 0.16932846520638076
Model: <catboost.core.CatBoostRegressor object at 0x7f83a19b1890>
MAE: 0.06882091297358045, RMSE: 0.16939961460871947
Model: <catboost.core.CatBoostRegressor object at 0x7f83a1eb2ad0>
MAE: 0.06881286643093604, RMSE: 0.16927206000034226
Model: <catboost.core.CatBoostRegressor object at 0x7f83a3184210>
MAE: 0.06881396167888595, RMSE: 0.16940058666603539
Model: <catboost.core.CatBoostRegressor object at 0x7f83a222dc50>
MAE: 0.06879691769152384, RMSE: 0.16933170558929114


#### Hyperparameter tuning for catboost


In [21]:
from sklearn.model_selection import RandomizedSearchCV
param_dist = { "learning_rate": np.linspace(0,0.2,5),
               "depth": [4,5,6,7,8,9,10],
             }
model = CatBoostRegressor(loss_function='MAE', eval_metric='MAE')
Grid_CBC = RandomizedSearchCV(model, param_dist, cv = 2, n_jobs=-1)
Grid_CBC.fit(X_train, y_train)    


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
2 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/catboost/core.py", line 5733, in fit
    save_snapshot, snapshot_file, snapshot_interval, init_model, callbacks, log_cout, log_cerr)
  File "/usr/local/lib/python3.7/dist-packages/catboost/core.py", line 2347, in _fit
    callbacks=callbacks
  File "/usr/local/lib/python3.

0:	learn: 0.0685643	total: 26.6ms	remaining: 26.5s
1:	learn: 0.0685197	total: 53.4ms	remaining: 26.6s
2:	learn: 0.0684725	total: 76.6ms	remaining: 25.5s
3:	learn: 0.0684371	total: 102ms	remaining: 25.4s
4:	learn: 0.0684044	total: 125ms	remaining: 24.8s
5:	learn: 0.0683726	total: 149ms	remaining: 24.8s
6:	learn: 0.0683467	total: 173ms	remaining: 24.5s
7:	learn: 0.0683190	total: 198ms	remaining: 24.6s
8:	learn: 0.0682978	total: 225ms	remaining: 24.7s
9:	learn: 0.0682731	total: 254ms	remaining: 25.1s
10:	learn: 0.0682567	total: 277ms	remaining: 24.9s
11:	learn: 0.0682429	total: 300ms	remaining: 24.7s
12:	learn: 0.0682263	total: 326ms	remaining: 24.8s
13:	learn: 0.0682165	total: 350ms	remaining: 24.6s
14:	learn: 0.0682070	total: 375ms	remaining: 24.6s
15:	learn: 0.0681991	total: 397ms	remaining: 24.4s
16:	learn: 0.0681877	total: 420ms	remaining: 24.3s
17:	learn: 0.0681783	total: 443ms	remaining: 24.2s
18:	learn: 0.0681660	total: 470ms	remaining: 24.3s
19:	learn: 0.0681584	total: 506ms	rema

RandomizedSearchCV(cv=2,
                   estimator=<catboost.core.CatBoostRegressor object at 0x7f000056b3d0>,
                   n_jobs=-1,
                   param_distributions={'depth': [4, 5, 6, 7, 8, 9, 10],
                                        'learning_rate': array([0.  , 0.05, 0.1 , 0.15, 0.2 ])})

In [22]:
print(Grid_CBC.best_params_)

{'learning_rate': 0.15000000000000002, 'depth': 4}


In [23]:
y_pred= Grid_CBC.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
print(f"Model: {Grid_CBC}")
print(f"MAE: {mae}, RMSE: {rmse}")

Model: RandomizedSearchCV(cv=2,
                   estimator=<catboost.core.CatBoostRegressor object at 0x7f000056b3d0>,
                   n_jobs=-1,
                   param_distributions={'depth': [4, 5, 6, 7, 8, 9, 10],
                                        'learning_rate': array([0.  , 0.05, 0.1 , 0.15, 0.2 ])})
MAE: 0.06880667577688462, RMSE: 0.16932846520638076


# Preparing Test Data

In [14]:
sample_submission_data

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0
...,...,...,...,...,...,...,...
2985212,168176230,0,0,0,0,0,0
2985213,14273630,0,0,0,0,0,0
2985214,168040630,0,0,0,0,0,0
2985215,168040830,0,0,0,0,0,0


In [15]:
original_test_df = pd.merge(sample_submission_data[['ParcelId']], properties_2017.rename(columns = {'parcelid': 'ParcelId'}), how = 'left', on = 'ParcelId')
original_test_df

,ParcelId,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,NaN,...,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,NaN,...,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2985212,168176230,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,110245.0,226900.0,2016.0,116655.0,3035.26,NaN,NaN,NaN
2985213,14273630,NaN,NaN,NaN,3.0,3.0,NaN,NaN,3.0,NaN,...,NaN,NaN,378466.0,762157.0,2016.0,383691.0,8653.10,NaN,NaN,NaN
2985214,168040630,NaN,NaN,NaN,2.5,3.0,NaN,NaN,2.5,NaN,...,NaN,NaN,201575.0,483188.0,2016.0,281613.0,5671.40,NaN,NaN,NaN
2985215,168040830,NaN,NaN,NaN,3.0,4.0,NaN,NaN,3.0,NaN,...,NaN,NaN,347863.0,578621.0,2016.0,230758.0,5894.36,NaN,NaN,NaN


In [16]:
additonal_test_columns_to_drop= ['propertycountylandusecode', 'propertyzoningdesc','ParcelId','airconditioningtypeid__12.0']
#additonal_test_columns_to_drop= ['propertycountylandusecode', 'propertyzoningdesc','ParcelId']
def test_data_processing(df):
  df = clean_data(df)
  df = df.drop(columns_to_drop_test,axis=1)
  df = one_hot_encoding(df)
  #print(df.columns)
  df = df.drop(additonal_test_columns_to_drop,axis=1)
  df = replace_nan(df) 

  return df

In [17]:
#original_test_df = original_test_df.drop(neg_cor,axis=1)
basic_test_df = test_data_processing(original_test_df)
basic_test_df

Cleaning Data

Perform One hot encoding

Replace Empty Values with 0



,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,garagecarcnt,garagetotalsqft,...,heatingorsystemtypeid__10.0,heatingorsystemtypeid__11.0,heatingorsystemtypeid__12.0,heatingorsystemtypeid__13.0,heatingorsystemtypeid__14.0,heatingorsystemtypeid__18.0,heatingorsystemtypeid__19.0,heatingorsystemtypeid__20.0,heatingorsystemtypeid__21.0,heatingorsystemtypeid__24.0
0,0.0,0.0,0.0,0.0,0.0,0.0,6037.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,6037.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,73026.0,0.0,6037.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,0.0,0.0,6.0,0.0,5068.0,0.0,6037.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,1776.0,0.0,6037.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2985212,0.0,0.0,0.0,0.0,0.0,0.0,6059.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2985213,3.0,3.0,0.0,3.0,2009.0,2009.0,6059.0,2.0,2.0,549.0,...,0,0,0,0,0,0,0,0,0,0
2985214,2.5,3.0,0.0,2.5,1390.0,1390.0,6059.0,2.0,2.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2985215,3.0,4.0,0.0,3.0,2381.0,2381.0,6059.0,3.0,2.0,442.0,...,0,0,0,0,0,0,0,0,0,0


### CatBoostx5 Model Prediction

In [18]:
Prediction_Dates = {'201610':'2016-10-01','201611':'2016-11-01','201612':'2016-12-01','201710':'2017-10-01','201711':'2017-11-01','201712':'2017-12-01'}
prediction_list = []
catBoost_models_sample_submission_data = sample_submission_data.copy()
for key,val in Prediction_Dates.items():
  print(key,val)
  test_df = basic_test_df.copy()
  test_df['transactiondate'] = pd.Timestamp(val) 
  test_df = transform_date(test_df)
  #test_df = test_df.loc[:, rfe_columns]
  print(test_df.shape)
  #predictions = model4.predict(test_df)
  #predictions2 = model7.predict(test_df)
  for i in catBoost_models:
    print(catBoost_models.index(i))
    prediction = i.predict(test_df)
    prediction_list.append(prediction)
  predictions = (prediction_list[0]+prediction_list[1]+prediction_list[2]+prediction_list[3]+prediction_list[4])/5
  catBoost_models_sample_submission_data[key] = [float(format(x, '.4f')) for x in predictions]

201610 2016-10-01
Transforming Date Information

(2985217, 62)
0
1
2
3
4
201611 2016-11-01
Transforming Date Information

(2985217, 62)
0
1
2
3
4
201612 2016-12-01
Transforming Date Information

(2985217, 62)
0
1
2
3
4
201710 2017-10-01
Transforming Date Information

(2985217, 62)
0
1
2
3
4
201711 2017-11-01
Transforming Date Information

(2985217, 62)
0
1
2
3
4
201712 2017-12-01
Transforming Date Information

(2985217, 62)
0
1
2
3
4


In [19]:
catBoost_models_sample_submission_data

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0.0113,0.0113,0.0113,0.0113,0.0113,0.0113
1,10759547,0.0196,0.0196,0.0196,0.0196,0.0196,0.0196
2,10843547,0.0219,0.0219,0.0219,0.0219,0.0219,0.0219
3,10859147,0.0265,0.0265,0.0265,0.0265,0.0265,0.0265
4,10879947,0.0225,0.0225,0.0225,0.0225,0.0225,0.0225
...,...,...,...,...,...,...,...
2985212,168176230,0.0185,0.0185,0.0185,0.0185,0.0185,0.0185
2985213,14273630,0.0108,0.0108,0.0108,0.0108,0.0108,0.0108
2985214,168040630,0.0206,0.0206,0.0206,0.0206,0.0206,0.0206
2985215,168040830,0.0355,0.0355,0.0355,0.0355,0.0355,0.0355


In [20]:
from google.colab import files
catBoost_models_sample_submission_data.to_csv('catBoost_submission.csv',index=False)
files.download("catBoost_submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>